In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from statsmodels.iolib.summary2 import summary_col

In [3]:
data_dir = Path() / "results"
figs_dir = Path() / "figs"
figs_dir.mkdir(exist_ok=True)

# Regression models

In [14]:
df = pd.read_csv(data_dir / "foundation_cond_agg.csv")

# renaming columns to use same terminology
df.rename(
    columns={
        "MFQ_harm": "MFQ_Care",
        "MFQ_ingroup": "MFQ_Loyalty",
        "MFQ_authority": "MFQ_Authority",
        "MFQ_purity": "MFQ_Purity",
        "MFQ_fairness": "MFQ_Fairness",
    },
    inplace=True,
)

df.head(2)

,agent,id,condition,MFQ_Care,MFQ_Fairness,MFQ_Loyalty,MFQ_Authority,MFQ_Purity,MFV_Care (e),MFV_Authority,MFV_Loyalty,MFV_Purity,MFV_Liberty,MFV_Care (p),MFV_Fairness
0,Claude 2.1,0,qv,4.166667,4.833333,2.333333,2.166667,1.333333,2.363636,2.5,2.4,3.571429,2.142857,3.555556,2.888889
1,Claude 2.1,1,qv,4.166667,4.833333,2.500000,1.833333,1.500000,3.000000,2.4,2.6,3.571429,2.714286,3.444444,2.888889


Applying StandardScaler

In [5]:
scaler = StandardScaler()

# apply on columns starting with MFQ or MFV
df.loc[
    :, df.columns.str.startswith("MFQ") | df.columns.str.startswith("MFV")
] = scaler.fit_transform(
    df.loc[:, df.columns.str.startswith("MFQ") | df.columns.str.startswith("MFV")]
)

In [20]:
# rename columns to standardize under care, loyalty etc

formula = "Q('{}') ~ MFQ_Authority + MFQ_Care + MFQ_Fairness + MFQ_Loyalty + MFQ_Purity"

dependents = [x for x in df.columns if x.startswith("MFV_")]
dependents.sort()

In [21]:
df.columns

Index(['agent', 'id', 'condition', 'MFQ_Care', 'MFQ_Fairness', 'MFQ_Loyalty',
       'MFQ_Authority', 'MFQ_Purity', 'MFV_Care (e)', 'MFV_Authority',
       'MFV_Loyalty', 'MFV_Purity', 'MFV_Liberty', 'MFV_Care (p)',
       'MFV_Fairness'],
      dtype='object')

# Models for Claude

In [22]:
models = list()

for i in dependents:
    mod = smf.ols(
        formula=formula.format(i),
        data=df.query("agent == 'Claude 2.1'"),
    )
    res = mod.fit()
    models.append(res)

output = summary_col(models, stars=True)

print(output)


               Q('MFV_Authority') Q('MFV_Care (e)') Q('MFV_Care (p)') Q('MFV_Fairness') Q('MFV_Liberty') Q('MFV_Loyalty') Q('MFV_Purity')
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept      3.7999***          4.3464***         5.0767***         4.0075***         4.2387***        3.9378***        4.9131***      
               (0.6718)           (0.7011)          (0.6775)          (0.6719)          (0.9466)         (0.8520)         (0.6648)       
MFQ_Authority  -0.0057            -0.0594           -0.1647           -0.1157           0.0852           0.0186           -0.1446        
               (0.1104)           (0.1152)          (0.1113)          (0.1104)          (0.1555)         (0.1400)         (0.1092)       
MFQ_Care       0.0475             0.0840            0.0112            0.0449            -0.0042          -0.0257          0.0160         
               (0.1121)          

In [23]:
models = list()

for i in dependents:
    mod = smf.ols(
        formula=formula.format(i),
        data=df.query("agent == 'GPT-4'"),
    )
    res = mod.fit()
    models.append(res)

output = summary_col(models, stars=True)

print(output)


               Q('MFV_Authority') Q('MFV_Care (e)') Q('MFV_Care (p)') Q('MFV_Fairness') Q('MFV_Liberty') Q('MFV_Loyalty') Q('MFV_Purity')
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept      3.9709***          2.9704***         4.4172***         4.1667***         3.1867***        2.8685***        5.0668***      
               (0.7119)           (0.8777)          (0.3286)          (0.4899)          (0.8854)         (0.8994)         (0.4205)       
MFQ_Authority  0.0420             -0.0141           -0.0136           -0.0199           -0.0980          -0.2746**        -0.0168        
               (0.0881)           (0.1087)          (0.0407)          (0.0607)          (0.1096)         (0.1114)         (0.0521)       
MFQ_Care       0.0929             0.0513            0.0702            0.1827*           -0.0137          0.3570**         0.1969**       
               (0.1354)          